In [5]:
# resources https://github.com/HKUDS/LightRAG

from bs4 import BeautifulSoup
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import nest_asyncio
import requests
import os

from llama_parse import LlamaParse
#nest_asyncio.apply()

In [6]:
#nest_asyncio.apply()
#from google.colab import userdata


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LLAMAPARSE_API_KEY = os.getenv('LLAMAPARSE_API_KEY')
ANTHROPIC_API_KEY = os.getenv("CLAUDE_API_KEY")

In [ ]:
import pandas as pd
import os

# Load the CSV with metadata
metadata_csv = "data/metadata.csv"
df = pd.read_csv(metadata_csv)

# Load articles from the data folder
data_folder = "data"
articles = []
for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):  # Assuming articles are stored as .txt files
        with open(os.path.join(data_folder, filename), "r", encoding="utf-8") as file:
            articles.append(file.read())

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"  # You can use other variants like "facebook/rag-token-nq"
tokenizer = RagTokenizer.from_pretrained(model_name)
retriever = RagRetriever.from_pretrained(model_name, index_name="custom")  # Use a custom index if needed
model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever)

In [ ]:
def filter_article(article_text):
    # Tokenize the input
    inputs = tokenizer(article_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate a summary or classification
    outputs = model.generate(input_ids=inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Define keywords to filter out
    unwanted_keywords = ["sports", "celebrity", "entertainment", "gossip", "movie", "music"]

    # Check if any unwanted keywords are in the generated text
    if any(keyword in generated_text.lower() for keyword in unwanted_keywords):
        return False  # Filter out this article
    else:
        return True  # Keep this article

In [7]:
embed_model = "local:BAAI/bge-small-en-v1.5" #https://huggingface.co/collections/BAAI/bge-66797a74476eb1f085c7446d

documents = " "

vector_index_std = VectorStoreIndex(documents, embed_model = embed_model)


llm_gpt35 = OpenAI(model="gpt-3.5-turbo", api_key = OPENAI_API_KEY)
query_engine_gpt35 = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_gpt35)

llm_gpt4o = OpenAI(model="gpt-4o", api_key = OPENAI_API_KEY)
query_engine_gpt4o = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_gpt4o)


tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

llm_claude = Anthropic(model="claude-3-5-sonnet-20240620", api_key=ANTHROPIC_API_KEY)
query_engine_claude = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_claude)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\AppData\Local\llama_index\models\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: nodes must be a list of Node objects.

In [5]:
nums = [[1,2,3],[4,5,6],[7,8,9]]

for rows in range(len(nums)):
	for columns in range(len(nums[rows])):
		print(nums[rows][columns])

1
2
3
4
5
6
7
8
9


In [ ]:
query1 = "Is this news story about politics, global events"
response = query_engine_gpt35.query(query1)
print("GPT-3.5 Turbo")
print(str(response))

In [ ]:
def query(query1, **kwargs):
	headline = kwargs.get("headline")
	tagline = kwargs.get("tagline")
	query1 = "Is this news story about politics, or global events?"
	response = query_engine_gpt35.query(query1)
	if response == "yes":
		return True


query(headline="Is this news story about politics, global events", tagline="this is the tagline")

Sacramento
